####Using Transformer LLMs for common tasks
1. Summarization
2. Translation
3. Question Answering
4. Table Question Answering
5. Fill Mask
6. Feature Extraction
7. Zero Shot Classification

In [0]:
from transformers import pipeline
import pandas as pd

2026-02-24 01:16:55.510673: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-24 01:16:55.511554: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-24 01:16:55.514659: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-24 01:16:55.523740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-24 01:16:55.538610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

####Translation
Translation is the task of converting text from one language to another.

Translate the given statement from english to hindi.\
You can use the Helsinki-NLP/opus-mt-en-hi model

In [0]:
statement = "This is a good book."
translator = pipeline(task="translation",model="Helsinki-NLP/opus-mt-en-hi")
result = translator(statement)


/databricks/python/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [0]:
print(result)

[{'translation_text': 'यह एक अच्छी किताब है.'}]


Translating to another Language (Telugu)

In [0]:
from transformers import pipeline

statement = "Humans are so cruel and selfish. No one cares about animals and the environment."

translator = pipeline(
    task="translation",
    model="facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn",
    tgt_lang="tel_Telu"
)

result = translator(statement)

print(result[0]["translation_text"])

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


మానవులు చాలా క్రూరమైనవారు, స్వార్థపరులు. జంతువులు, పర్యావరణం ఎవరికీ పట్టించుకోవు.


####Question Answering
Question Answering models can retrieve the answer to a question from a given text, which is useful for searching for an answer in a document. Some question answering models can generate answers without context!

In [0]:
question = "Where do I live?"
context = "My name is ChatGpt, I became very popular now a days and I can answer your questions and I live in Mars."
qa_pipeline = pipeline(task="question-answering")


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [0]:
result = qa_pipeline(question = question, context = context)

In [0]:
print(result)

{'score': 0.9587178230285645, 'start': 99, 'end': 103, 'answer': 'Mars'}


In [0]:
context_ = """Tara and Sitara were two ducks. They were best friends. They lived in a beautiful pond.
Tara loved to eat small fishes, Tare really likes to eat fishes. Sitara loved swimming. They ate together. They
spent their whole day enjoying with each other.
"""

In [0]:
questions = ["What did Tara love?","Find out a describing word used for ‘pond’ in the story.","Where did Tara and Sitara live?"]
for ques_ in questions:
    result = qa_pipeline(question = ques_, context = context_)
    print(result)

{'score': 0.4285235106945038, 'start': 99, 'end': 118, 'answer': 'to eat small fishes'}
{'score': 0.09524255990982056, 'start': 72, 'end': 86, 'answer': 'beautiful pond'}
{'score': 0.5063747763633728, 'start': 70, 'end': 86, 'answer': 'a beautiful pond'}
